In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from hansenhandler import HansenHandler as hh
import os

In [2]:
latlon = [(hh.lat_num2str(lat), hh.long_num2str(long))
            for lat in range(-50, 90, 10)
            for long in range(-180, 180, 10)]

filenames = [f"data/interim/tile_aggregates/tile_stats_{lat}_{lon}.csv"
                for lat, lon in latlon]

dfs = [pd.read_csv(f, index_col=0) for f in filenames
        if os.path.exists(f)]

missing = [latlon[i] for i, f in enumerate(filenames) if not os.path.exists(f)]

print(len(missing))

0


In [3]:
country_renames = pd.read_csv("data/interim/country_renames.csv")
year_renames = {f"loss_20{x+1}": f"loss_20{x+1:02d}" for x in range(20)}


cover_cat_df = (
    pd.concat(dfs)
        # Rename e.g. loss_201 to loss_2001
        .rename(year_renames, axis="columns")
        .merge(country_renames)
        .drop(columns=["minx","maxx","miny","maxy","group","country"])
        .rename(columns={"country_new":"country"})

        # Drop rows from tiles with no data
        .dropna(subset=["cover_cat"])

        # Drop country-tropic pairs with no observations
        .groupby(["code", "tropics"])
        .filter(lambda x: x.pixels.sum() > 0)
        .assign(pixels_sub_25 = lambda x: np.where(x.cover_cat == "1-25", x.pixels, 0))

        # Sum across different tiles
        .groupby(["code", "tropics", "cover_cat", "country"])
        .agg(np.sum)
        .reset_index()

        # Pivot longer
        .pipe(pd.wide_to_long, stubnames="loss", sep="_", i=["code","tropics","cover_cat"], j="year")
        .reset_index()
        .sort_values(["code","tropics","year","cover_cat"])

        # Change units from m^2 to km^2
        .assign(
            cover_2000  = lambda x: x.cover_2000 / 1000**2,
            area        = lambda x: x.area / 1000**2,
            loss        = lambda x: x.loss / 1000**2 / 100 # TODO change with next run
        )

)

cover_cat_df.to_csv("data/clean/hansen_by_binned_cover.csv")

In [4]:
cover_cat_df

,code,tropics,cover_cat,year,area,country,pixels,pixels_sub_25,cover_2000,loss
0,ABW,True,0,2001,170.495936,Aruba,226867.0,0.0,0.000000,0.000000
20,ABW,True,1-25,2001,7.480603,Aruba,9953.0,9953.0,0.463355,0.000000
40,ABW,True,26-50,2001,0.250280,Aruba,333.0,0.0,0.077263,0.000000
60,ABW,True,51-75,2001,0.045844,Aruba,61.0,0.0,0.028175,0.000000
80,ABW,True,76-100,2001,0.118001,Aruba,157.0,0.0,0.098212,0.000000
...,...,...,...,...,...,...,...,...,...,...
25119,ZWE,True,0,2020,55776.044302,Zimbabwe,76898298.0,0.0,0.000000,0.000000
25139,ZWE,True,1-25,2020,307094.170788,Zimbabwe,421580268.0,421580268.0,30972.914752,13.659244
25159,ZWE,True,26-50,2020,21172.274502,Zimbabwe,28982859.0,0.0,7267.685804,16.373349
25179,ZWE,True,51-75,2020,2132.713296,Zimbabwe,2932930.0,0.0,1254.261651,12.106060


In [66]:
forest_threshold = 50

df = (
    cover_cat_df
        # Define forest as above threshold tree cover
        .rename(columns={"cover_2000":"treecover_2000","loss":"treecover_loss"})
        .assign(
            cover_cat_max = lambda x: x.cover_cat.str.split("-").apply(lambda l: max([int(n) for n in l])),
            cover_2000 = lambda x: np.where(x.cover_cat_max <= forest_threshold, 0, x.treecover_2000),
            loss = lambda x: np.where(x.cover_cat_max <= forest_threshold, 0, x.treecover_loss)
        )

        # Aggregate over treecover bins
        .drop(columns="cover_cat")
        .groupby(["code","tropics","country","year"])
        .agg(np.sum)
        .reset_index()

        # Calculate % of tree-cover below forest threshold
        .assign(sub_25_cover_pct = lambda x: x.pixels_sub_25 / x.pixels * 100)
)

df.to_csv("data/clean/hansen.csv")